# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [7]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [8]:
vacation_data = pd.read_csv("weather_data.csv")
vacation_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,tuktoyaktuk,69.45,-133.04,-13.00,75,20,9.17,CA,1607546565
1,klaksvik,62.23,-6.59,44.60,100,100,13.87,FO,1607546770
2,tuatapere,-46.13,167.68,62.01,80,100,1.01,NZ,1607546770
3,pustoshka,56.34,29.37,22.08,94,0,8.95,RU,1607546770
4,tautira,-17.73,-149.15,84.20,66,75,8.05,PF,1607546618
...,...,...,...,...,...,...,...,...,...
103,torbay,47.67,-52.73,39.00,93,75,17.22,CA,1607546786
104,lebu,-37.62,-73.65,58.69,66,0,16.17,CL,1607546786
105,walvis bay,-22.96,14.51,64.00,90,0,7.61,NaN,1607546786
106,kaeo,-35.10,173.78,72.00,73,6,11.01,NZ,1607546787


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [9]:
gmaps.configure(api_key=g_key)

locations = vacation_data[["Lat", "Lng"]].astype(float)
humidity = vacation_data["Humidity"].astype(float)
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [10]:
new_vacation = vacation_data.loc[(vacation_data["Max Temp"] < 80) & (vacation_data["Max Temp"] > 70) & (vacation_data["Wind Speed"] < 10) 
                                 & (vacation_data["Cloudiness"] == 0)] 
new_vacation.dropna().head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
43,chuy,-33.70,-53.46,72.25,69,0,9.80,UY,1607546572
85,tokonou,9.65,-9.78,76.19,32,0,2.28,GN,1607546783


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [16]:
hotel_df = new_vacation.loc[:,["City","Country", "Lat", "Lng"]]
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Lat,Lng,Hotel Name
43,chuy,UY,-33.70,-53.46,
85,tokonou,GN,9.65,-9.78,


In [12]:
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"keyword": "hotel", "type": "hotel", "radius": 5000, "key": g_key}

#iterate through hotel dataframe
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    params["location"] = f'{lat}, {lng}'
    
    #assemble URL and API request
    print(f'Retrieving results for {index}: {city_name}.')
    response = requests.get(url, params=params).json()
    
    results = response["results"]
    
    try:
        print(f'The closest hotel in {city_name} is {results[0]["name"]}.')
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]
          
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")   
    
    time.sleep(1)
          
print("----------------------------")                               

Retrieving results for 43: chuy.
The closest hotel in chuy is Turis Firper Hotel.
Retrieving results for 85: tokonou.
Missing field/result... skipping.
----------------------------


In [13]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
43,chuy,UY,-33.70,-53.46,Turis Firper Hotel
85,tokonou,GN,9.65,-9.78,


In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [15]:
# Add marker layer on top of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))